You know it's going to be a good notebook when it starts with the following import:

In [1]:
from functools import cache

### Problem 1
Given an unlimited supply of coins of given denominations (e.g. unlimited pennies and nickels), find the minimum number of coins required to get the desired change.

In [ ]:
def n_coins(denominations, change):
    denominations = list(set(denominations))
    @cache
    def n_coins_cached(change):
        if change == 0:
            return 0
        elif change < 0:
            return float('inf')
        else:
            return min(
                1 + n_coins_cached(change - d)
                for d in denominations
            )
    return n_coins_cached(change)

In [9]:
assert n_coins([1, 5], 12) == 4
assert n_coins([1, 3, 5, 7], 15) == 3
assert n_coins([1], 1000) == 1000
assert n_coins([10, 25], 51) == float('inf')

TypeError: unhashable type: 'list'

The runtime is $O(dc)$ where $d$ is the number of denominations and $c$ is the amount of change that we want.

### Problem 2
Given a string and a list of valid words, determine if the string can be segmented into valid words.

In [4]:
def segmentable(valid_words, s):
    @cache
    def segmentable_cached(i):
        '''whether s[:i] is segmentable'''
        if i == 0:
            return True
        else:
            return any(
                segmentable_cached(i - len(valid_word))
                for valid_word in valid_words
                if s[:i].endswith(valid_word) # Python will cache the splice
            )

    return segmentable_cached(len(s))

In [5]:
s = 'penpineappleapplepen'
assert segmentable({'pen', 'pineapple', 'apple'}, s)
assert segmentable({'pen', 'pine', 'apple'}, s)
assert not segmentable({'pen', 'apple'}, s)
assert not segmentable({'pen', 'pineapple'}, s)

The runtime is $O(v\ell s)$ where $v$ is the number of valid words, $\ell$ is the (average) length of the valid words, and $s$ is the length of the given string.

### Problem 3

Let $A$ and $B$ be two lists of numbers. Determine the longest common (not necessarily contiguous) subsequence. For example, the answer for $A = [1, 3, 2, 4, 7]$ and $B = [5, 1, 3, 4, 9]$ is $[1, 3, 4].$

In [ ]:
def LCS(A, B):
    @cache
    def LCS_cached(i, j):
        if i == 0 or j == 0:
            return []
        elif A[i - 1] == B[j - 1]:
            return LCS_cached(i - 1, j - 1) + [A[i - 1]]
        else:
            LCS_A_discarded = LCS(i - 1, j)
            LCS_B_discarded = LCS(i, j - 1)
            return (
                LCS_A_discarded
                if len(LCS_A_discarded) > len(LCS_B_discarded)
                else LCS_B_discarded
            )

In [6]:
def LCS(A, B):
    @cache
    def len_LCS(i, j):
        '''The length of the LCS for A[:i] and B[:j]'''
        if i == 0 or j == 0:
            return 0
        elif A[i - 1] == B[j - 1]:
            return 1 + len_LCS(i - 1, j - 1)
        else:
            return max(len_LCS(i - 1, j), len_LCS(i, j - 1))
    
    i, j = len(A), len(B)
    answer = []
    while i > 0 and j > 0:
        if (x := A[i - 1]) == B[j - 1]:
            answer.append(x)
            i -= 1
            j -= 1
        elif len_LCS(i - 1, j) >= len_LCS(i, j - 1):
            i -= 1
        else:
            j -= 1
    return list(reversed(answer))

In [7]:
assert LCS([1, 3, 2, 4, 7], [5, 1, 3, 4, 9]) == [1, 3, 4]
assert LCS([0, 1], [0, 1]) == [0, 1]
assert LCS([2], [0, 1, 2, 3, 4]) == [2]

The runtime is $O(ab)$ where $a$ and $b$ are the lengths of $A$ and $B$ respectively.